In [1]:
import warnings
import time
warnings.simplefilter('ignore')

import logging
logging.basicConfig(format='%(levelname)-8s [%(asctime)s] (%(module)s : %(funcName)s) %(message)s',level=logging.INFO)
logging.info("application started")
logging.info("")
logging.info("")
# будем отображать графики прямо в jupyter'e
%pylab inline
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

import pandas as pd
import seaborn as sns

sns.set(rc=rcParams)
sns.set_style("darkgrid", {
    "axes.facecolor": "#282828",
    "axes.edgecolor":"#282828",
    "grid.color":"#298F4A",
    "xtick.color":"1",
    "ytick.color":"1",
    "text.color":"1",
    "axes.labelcolor":"1",
    "figure.facecolor":"#282828"
})
sns.set_palette(sns.color_palette("BrBG", 7))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

INFO     [2017-07-14 14:16:08,112] (<ipython-input-1-e86ed4fc6502> : <module>) application started
INFO     [2017-07-14 14:16:08,115] (<ipython-input-1-e86ed4fc6502> : <module>) 
INFO     [2017-07-14 14:16:08,118] (<ipython-input-1-e86ed4fc6502> : <module>) 


Populating the interactive namespace from numpy and matplotlib


In [58]:
import os
import re

In [349]:
def process(x):
    x=str(x).replace(',','.')
    
    if str(x)[-1]=='%':
        x=x[:-1]
    try:
        a=float(x)/100
        if a>1:
            a=a/100
        return a
    except Exception as e:
            return 0.0
        
may_june=pd.DataFrame()
for (root,dirs,files) in os.walk('./data/IFT results/may-june/'):
    for file in files:
        day=pd.read_csv(root+"/"+file,delimiter="\t",)
        day['date']=str(file[:-4])
        may_june=may_june.append(day)
april_may=pd.DataFrame()
for (root,dirs,files) in os.walk('./data/IFT results/april-may'):
    for file in files:
        day=pd.read_csv(root+"/"+file,delimiter="\t",)
        day['date']=str(file[:-4])
        april_may=april_may.append(day)

april_may=april_may.reset_index()
del april_may['index']

may_june=may_june.reset_index()
del may_june['index']
def asd(x):
    if x[0] in '0123456789':
        x=x[6:]
    x=x.strip()
    if "Цифровые продажи" in x:
        return "Цифровые продажи" 
    elif "Уведомления" in x:
        return "Уведомления"
    else :
        return x
april_may.project=april_may.project.apply(asd)
may_june.project=may_june.project.apply(asd)

april_may=april_may[april_may.project!='SberPay (ex E-Инвойсинг)']
april_may['date']=pd.to_datetime(april_may['date'],yearfirst=True,dayfirst=True)
may_june['date']=pd.to_datetime(may_june['date'],yearfirst=True,dayfirst=True)
# april_may.head()

may_june.percent=may_june.percent.apply(process)
# may_june
# april_may.sort('project').groupby('project').count()
# may_june.sort('project').groupby('project').count()

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7ff1d9753a58>>
Traceback (most recent call last):
  File "/home/azaz/DISTR/xgboost/python-package/xgboost/core.py", line 343, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'
/home/azaz/anaconda3/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-07-14 16:09:28.659295. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [321]:
# pd.get_dummies(april_may,columns=['project']).head()
april_may_cat=pd.get_dummies(april_may,columns=['project'])
may_june_cat=pd.get_dummies(may_june,columns=['project'])

In [343]:
full = pd.DataFrame.append(april_may.rename(columns={'процент':'percent'}),may_june)
aaa=full.set_index('project')
gr=aaa.groupby(aaa.index)
X=[]
Y=[]
for name,group in gr:
    first=True
    for x in group.sort('date')['percent']:
        X.append(x)    
        if first:
            Y.append(0)
            first=False
        else:
            Y.append(X[-1])


/home/azaz/anaconda3/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-07-14 16:07:31.520646. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [348]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error,roc_auc_score

X=np.array(X)
X=X.reshape((X.shape[0],1))
Y=np.array(Y)
Y=Y.reshape((Y.shape[0],1))

X.shape
Y.shape

trainX,testX,trainY,testY=train_test_split(X,Y,test_size=0.3,random_state=123)

dtrain = xgb.DMatrix(trainX,trainY)
dtest = xgb.DMatrix(testX,testY)
bst = xgb.train({"eta":0.1,"max_depth":3,"subsample":0.5},dtrain,num_boost_round=30)
# xgb.plot_importance(bst)
mean_squared_error(bst.predict(dtrain),trainY)
mean_squared_error(bst.predict(dtest),testY)
list(zip(bst.predict(dtest),testY))

(1267, 1)

(1267, 1)

0.00052317708538094255

0.00025950624513531825

[(0.5499717, array([ 0.55])),
 (0.97570288, array([ 1.])),
 (0.62392169, array([ 0.636])),
 (0.028421134, array([ 0.])),
 (0.38496143, array([ 0.384])),
 (0.06701079, array([ 0.049])),
 (0.19195864, array([ 0.1906])),
 (0.39612788, array([ 0.3906])),
 (0.52501631, array([ 0.52])),
 (0.4875159, array([ 0.49])),
 (0.97570288, array([ 1.])),
 (0.24326527, array([ 0.25])),
 (0.8345508, array([ 0.85])),
 (0.67053592, array([ 0.68])),
 (0.028421134, array([ 0.])),
 (0.033039868, array([ 0.])),
 (0.43059632, array([ 0.4256])),
 (0.70237195, array([ 0.7146])),
 (0.1891351, array([ 0.183])),
 (0.43059632, array([ 0.4293])),
 (0.97386223, array([ 0.993])),
 (0.028421134, array([ 0.])),
 (0.64543891, array([ 0.64])),
 (0.35151571, array([ 0.339])),
 (0.52501631, array([ 0.52])),
 (0.24326527, array([ 0.25])),
 (0.52501631, array([ 0.5217])),
 (0.045722187, array([ 0.03])),
 (0.033039868, array([ 0.018])),
 (0.97570288, array([ 1.])),
 (0.82495809, array([ 0.8398])),
 (0.028421134, array([ 0.])),


/home/azaz/anaconda3/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-07-14 16:09:04.389760. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
